In [455]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup as bts
import pandas as pd
import re
import time
import sys
from datetime import datetime
from datetime import date
import time
import re

In [7]:
def getAndParseURL(url):
    result = requests.get(url,headers={"User-Agent":"Mozilla/5.0"})
    soup = bts(result.text, 'html.parser')
    return soup

In [359]:
url = "https://www.imdb.com/chart/top"
soup = getAndParseURL(url)
movie_urls = []
for i in soup.tbody.findAll('tr'):
    movie_urls.append('https://www.imdb.com'+i.findAll('td')[1].a['href'])

In [474]:
row = []
for i in movie_urls:
    
    try:
        html = getAndParseURL(i)
    except:
        print("Couldn't take the data1")

    try:
        title = html.find('h1', {"data-testid":"hero-title-block__title"}).text
    except:
        print("Couldn't take the data2")

    try:
        wwgross = int(''.join(html.find('li', {"data-testid":"title-boxoffice-cumulativeworldwidegross"}).text.split('$')[1].split(',')))
    except:
        print("Couldn't take the data3")

#     budget = int(''.join(html.find('li', {"data-testid":"title-boxoffice-budget"}).text.split('$')[1].split(',')).split(' ')[0])
#     budget = int(''.join(html.find('li', {"data-testid":"title-boxoffice-budget"}).li.span.text.split(' ')[0].split(','))[1:])
#     budget = int(''.join(html.find('li', {"data-testid":"title-boxoffice-budget"}).li.span.text.split(' ')[0].split(',')).split('$')[1])
    try:
        budget = int(re.findall("\d+", ''.join(html.find('li', {"data-testid":"title-boxoffice-budget"}).li.span.text.split(' ')[0].split(',')))[0])
    except:
        print("Couldn't take the data4")

    try:
        releaseDate = int(html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[0].span.text)
    except:
        print("Couldn't take the data5")

    try:
        certificate = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[1].span.text
    except:
        print("Couldn't take the data6")

    try:
        runtime = html.find('ul', {"data-testid":"hero-title-block__metadata"}).findAll('li')[2].text.split(' ')
        runtime = int(runtime[0].split('h')[0]) * 60 + int(runtime[1].split('m')[0])
    except:
        print("Couldn't take the data7")

    try:
        score = float(html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating__score"}).text.split('/')[0])
    except:
        print("Couldn't take the data8")

    try:
        scoreN = html.find('div', {"data-testid":"hero-rating-bar__aggregate-rating"}).a.div.div.findAll('div')[1].findAll('div')[2].text
        if 'M' in scoreN:
            scoreNumber = int(float(scoreN.split('M')[0])*1_000_000)
        elif 'K' in scoreN:
            scoreNumber = int(float(scoreN.split('K')[0])*1_000_000)
        else:
            scoreNumber = int(scoreN)
    except:
        print("Couldn't take the data9")

    try:
        iR = html.find('ul', {"data-testid":"reviewContent-all-reviews"}).findAll('li')[0].span.span.text
        if 'K' in iR:
            imdbReviews = int(float(iR.split('K')[0]))
        else:
            imdbReviews = int(iR)
    except:
        print("Couldn't take the data10")

    try:
        criticReviews = int(html.find('ul', {"data-testid":"reviewContent-all-reviews"}).findAll('li')[1].span.span.text)
    except:
        print("Couldn't take the data11")

    try:
        metaScore = int(html.find('ul', {"data-testid":"reviewContent-all-reviews"}).findAll('li')[2].span.span.text)
    except:
        print("Couldn't take the data12")

    try:
        directors = [] # director = html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[0].text.split('Director')[1]
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[0].findAll('a'):
            directors.append(i.text)
    except:
        print("Couldn't take the data13")

    try:
        writers = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[1].findAll('a'):
            writers.append(i.text)
    except:
        print("Couldn't take the data14")

    try:
        stars = []
        for i in html.find('div', {"data-testid":"title-pc-wide-screen"}).findAll('li', {"data-testid":"title-pc-principal-credit"})[2].find('ul', {"role":"presentation"}).findAll('a'):
            stars.append(i.text)
    except:
        print("Couldn't take the data15")

    try:
        genres = []
        for i in html.find('li', {"data-testid":"storyline-genres"}).findAll('li'):
            genres.append(i.text)
    except:
        print("Couldn't take the data16")

    try:
        country = html.find('li', {"data-testid":"title-details-origin"}).a.text
    except:
        print("Couldn't take the data17")
    
    try:
        locations = []
        for i in html.find('li', {"data-testid":"title-details-filminglocations"}).findAll('a'):
            locations.append(i.text)
        locations = locations[1:-1]
    except:
        print("Couldn't take the data18")

    try:
        companies = []
        for i in html.find('li', {"data-testid":"title-details-companies"}).findAll('a'):
            companies.append(i.text)
        companies = companies[1:-1]
    except:
        print("Couldn't take the data19")

    row.append([title, wwgross, budget, releaseDate, certificate, runtime, score, scoreNumber, imdbReviews,
                   criticReviews, metaScore, directors, writers, stars, genres, country, companies])
    
    print(title, "...Done✔")
    print("Sleeping 5 seconds...")
    time.sleep(5)

column = ["title", "wwgross", "budget", "releaseDate", "certificate", "runtime", "score",
          "scoreNumber", "imdbReviews", "criticReviews", "metaScore", "directors", "writers",
          "stars", "genres", "country", "companies"]
dfMovies = pd.DataFrame.from_records(row, columns=column)

The Shawshank Redemption ...Done✔
Sleeping 5 seconds...
The Godfather ...Done✔
Sleeping 5 seconds...
The Godfather: Part II ...Done✔
Sleeping 5 seconds...
The Dark Knight ...Done✔
Sleeping 5 seconds...
12 Angry Men ...Done✔
Sleeping 5 seconds...
Schindler's List ...Done✔
Sleeping 5 seconds...
The Lord of the Rings: The Return of the King ...Done✔
Sleeping 5 seconds...
Pulp Fiction ...Done✔
Sleeping 5 seconds...
Il buono, il brutto, il cattivo ...Done✔
Sleeping 5 seconds...
The Lord of the Rings: The Fellowship of the Ring ...Done✔
Sleeping 5 seconds...
Fight Club ...Done✔
Sleeping 5 seconds...
Forrest Gump ...Done✔
Sleeping 5 seconds...
Inception ...Done✔
Sleeping 5 seconds...
The Lord of the Rings: The Two Towers ...Done✔
Sleeping 5 seconds...
Star Wars: Episode V - The Empire Strikes Back ...Done✔
Sleeping 5 seconds...
The Matrix ...Done✔
Sleeping 5 seconds...
Goodfellas ...Done✔
Sleeping 5 seconds...
One Flew Over the Cuckoo's Nest ...Done✔
Sleeping 5 seconds...
Shichinin no samurai

Lock, Stock and Two Smoking Barrels ...Done✔
Sleeping 5 seconds...
Shutter Island ...Done✔
Sleeping 5 seconds...
No Country for Old Men ...Done✔
Sleeping 5 seconds...
Couldn't take the data12
The Gold Rush ...Done✔
Sleeping 5 seconds...
Dial M for Murder ...Done✔
Sleeping 5 seconds...
The Thing ...Done✔
Sleeping 5 seconds...
Det sjunde inseglet ...Done✔
Sleeping 5 seconds...
Three Billboards Outside Ebbing, Missouri ...Done✔
Sleeping 5 seconds...
The Elephant Man ...Done✔
Sleeping 5 seconds...
Couldn't take the data3
Couldn't take the data18
Klaus ...Done✔
Sleeping 5 seconds...
The Sixth Sense ...Done✔
Sleeping 5 seconds...
Couldn't take the data4
Smultronstället ...Done✔
Sleeping 5 seconds...
Dune: Part One ...Done✔
Sleeping 5 seconds...
The Truman Show ...Done✔
Sleeping 5 seconds...
Couldn't take the data4
The Third Man ...Done✔
Sleeping 5 seconds...
Jurassic Park ...Done✔
Sleeping 5 seconds...
Salinui chueok ...Done✔
Sleeping 5 seconds...
V for Vendetta ...Done✔
Sleeping 5 seconds..

In [478]:
dfMovies

,title,wwgross,budget,releaseDate,certificate,runtime,score,scoreNumber,imdbReviews,criticReviews,metaScore,directors,writers,stars,genres,country,companies
0,The Shawshank Redemption,28817291,25000000,1994,13+,142,9.3,2500000,9,193,80,[Frank Darabont],"[Stephen King, Frank Darabont]","[Tim Robbins, Morgan Freeman, Bob Gunton]",[Drama],United States,[Castle Rock Entertainment]
1,The Godfather,246120986,6000000,1972,18+,175,9.2,1700000,4,269,100,[Francis Ford Coppola],"[Mario Puzo, Francis Ford Coppola]","[Marlon Brando, Al Pacino, James Caan]","[Crime, Drama]",United States,"[Paramount Pictures, Alfran Productions]"
2,The Godfather: Part II,48035783,13000000,1974,18+,202,9.0,1200000,1,190,90,[Francis Ford Coppola],"[Francis Ford Coppola, Mario Puzo]","[Al Pacino, Robert De Niro, Robert Duvall]","[Crime, Drama]",United States,"[Paramount Pictures, The Coppola Company, Amer..."
3,The Dark Knight,1005973645,185000000,2008,16+,152,9.0,2500000,7,433,84,[Christopher Nolan],"[Writers, Jonathan Nolan, Christopher Nolan, D...","[Christian Bale, Heath Ledger, Aaron Eckhart]","[Action, Crime, Drama, Thriller]",United States,"[Warner Bros., Legendary Entertainment, Syncopy]"
4,12 Angry Men,955,350000,1957,15+,96,9.0,739000000,1,160,96,[Sidney Lumet],[Reginald Rose],"[Henry Fonda, Lee J. Cobb, Martin Balsam]","[Crime, Drama]",United States,[Orion-Nova Productions]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,Le notti di Cabiria,755319,800000,1957,Not Rated,110,8.1,47000000,119,105,96,[Federico Fellini],"[Writers, Federico Fellini, Ennio Flaiano, Tul...","[Giulietta Masina, François Périer, Franca Marzi]",[Drama],Italy,"[Dino de Laurentiis Cinematografica, Les Films..."
246,The Princess Bride,30902442,16000000,1987,PG,98,8.1,413000000,927,227,77,[Rob Reiner],[William Goldman],"[Cary Elwes, Mandy Patinkin, Robin Wright]","[Adventure, Family, Fantasy, Romance]",United Kingdom,"[Act III Communications, Buttercup Films Ltd.,..."
247,"Paris, Texas",2207548,1162000,1984,13A,145,8.1,101000000,286,110,78,[Wim Wenders],"[L.M. Kit Carson, Sam Shepard, Walter Donohue]","[Harry Dean Stanton, Nastassja Kinski, Dean St...",[Drama],West Germany,"[Road Movies Filmproduktion, Argos Films, West..."
248,96,616333,1162000,2018,Not Rated,158,8.5,27000000,472,22,78,[C. Prem Kumar],[C. Prem Kumar],"[Vijay Sethupathi, Adithya Bhaskar, Trisha Kri...","[Drama, Romance]",India,[Madras Enterprises]


In [461]:
html = getAndParseURL("https://www.imdb.com/title/tt0317248/")
budget = int(re.findall("\d+", ''.join(html.find('li', {"data-testid":"title-boxoffice-budget"}).li.span.text.split(' ')[0].split(',')))[0])
budget

3300000